# Writing Tables Into Re-Usable Json Files

This notebook demonstrate how to use PMP's Json class to write easily parsable and reusable json files. See [this notebook](ReadInJsonFiles.ipynb) to see how to take advantage of this json format.

## Key Concepts


### Structure

This essentialy helps storing possibly complex tables into a json format that can later be easily parsed back into cdms/numpy variables.

The idea is that the user ran a set of metrics looping over different parameters and wants to store these results

For example for a given set of ***models***, loop through a given set of ***variables*** and for each variable compute a set of ***statitics***.

`model`, `variable` and `statistic` would represent what the call the json file's **structure**

Another example is to loop through model and realizations test against a set of references loop through modes and seasons to produce a statistic

Here the structure would be:

`model`, `realization`, `reference`, `mode`, `season`, `statistic`

A python code to generate this would probably look similar to this:

```python
for model in ["A", "B", "C"]:
    for realization in ["a", "b", "c", "d"]:
        for reference in ["ref1", "ref2"]:
            for mode in ["NAM", "NAO", "NPGO", "PDO", "PNA"]:
                for season in ["DJF", "JJA", "MAM"]:
                    for stat in ["rms", "average"]:
                        value = compute_some_stat(model, realization, reference, mode, season, stat)
```

### Dictionary

If stored in an array the final shape would be: `(3,4,2, 5, 3, 2)` which is 720 values

But in reality maybe for each mode the user runs a different set of statistics these can also depend on the variable. Storing this in an array would end up with a lot of missing values. This is not necessary when using dictionaries.

(If your data comes as a cdms2 variable, our package comes with a utility function to convert it back to a dictionary)


As described above the "Structure" defines what each layer of keys represent

In the example above to access the first value one would do:

```python

value = results["A"]["a"]["ref1"]["NAM"]["DJF"]["rms"]

```

Additional the "results" are expected to be in a filed named "RESULTS"

## Example


In [1]:
results = {"RESULTS": {"A": {"rms": .2, "mean":.5}, "B": {"mean":.123, "rms": .67}}}

import pcmdi_metrics

out = pcmdi_metrics.io.base.Base(".", "myfile.json")
out.write(results, json_structure=["model", "Statisitc"])

INFO::2019-05-23 14:00::pcmdi_metrics:: Results saved to a json file: /1TB/git/pcmdi_metrics/doc/jupyter/Jsons/myfile.json


In [2]:
!more myfile.json

{"RESULTS": {"A": {"rms": 0.2, "mean": 0.5}, "B": {"mean": 0.123, "rms": 0.67}},
 "json_version": 3.0, "json_structure": ["model", "Statisitc"], "provenance": {"
platform": {"OS": "Linux", "Version": "4.15.0-50-generic", "Name": "drdoom"}, "u
serId": "doutriaux1", "osAccess": false, "commandLine": "/1Tb/miniconda3/envs/ju
pyter-vcdat/lib/python3.6/site-packages/ipykernel_launcher.py -f /run/user/1000/
jupyter/kernel-76cecce7-1761-432d-915f-fc0bfd45647d.json", "date": "2019-05-23 1
4:00:21", "conda": {}, "packages": {}, "openGL": {"GLX": {"server": {}, "client"
: {}}}}}


In [3]:
J = pcmdi_metrics.io.base.JSONs(files=["myfile.json",], oneVariablePerFile=False)

In [4]:
J.getAxisList()

[   id: model
    Length: 2
    First:  A
    Last:   B
    Python id:  0x7f18d1163a90,    id: Statisitc
    Length: 2
    First:  mean
    Last:   rms
    Python id:  0x7f18d1163160]

In [5]:
J()

variable_5
masked_array(
  data=[[0.5  , 0.2  ],
        [0.123, 0.67 ]],
  mask=False,
  fill_value=1e+20)